In [1]:
import numpy as np
from sklearn.datasets import load_digits
# from sklearn.datasets import fetch_mldata
from collections import OrderedDict
from GHSOM import GHSOM
from matplotlib import pyplot as plt
# %matplotlib widget
%matplotlib inline


In [ ]:
data_shape = 8

# def __gmap_to_matrix(gmap):
#     gmap = gmap[0]
#     map_row = data_shape * gmap.shape[0]
#     map_col = data_shape * gmap.shape[1]
#     _image = np.empty(shape=(map_row, map_col), dtype=np.float32)
#     for i in range(0, map_row, data_shape):
#         for j in range(0, map_col, data_shape):
#             neuron = gmap[i // data_shape, j // data_shape]
#             _image[i:(i + data_shape), j:(j + data_shape)] = np.reshape(neuron, newshape=(data_shape, data_shape))
#     return _image

def __gmap_to_matrix(gmap, dataset, labels):
    gmap = gmap[0]
    # マップの横幅
    map_row = data_shape * gmap.shape[0]
    # マップの縦幅
    map_col = data_shape * gmap.shape[1]
    # 各要素に表示させる画像の初期化
    _image = np.empty(shape=(map_row, map_col), dtype=np.float32)
    mapping = [[list() for _ in range(gmap.map_shape()[1])]
        for _ in range(gmap.map_shape()[0])]
    for idx, label in enumerate(labels):
        winner_neuron = gmap.winner_neuron(dataset[idx])[0][0]
        r, c = winner_neuron.position
        mapping[r][c].append(label)

    for i in range(0, map_row, data_shape):
        for j in range(0, map_col, data_shape):

            # 各要素に表示させるニューロンの取得
            neuron = gmap[i // data_shape, j // data_shape]
            color = set(mapping[i // data_shape, j // data_shape])
            # ニューロンを変形させて画像として表示
            _image[i:(i + data_shape), j:(j + data_shape)
                   ] = np.reshape(neuron, newshape=(data_shape, data_shape))
    return _image

def __plot_child_with_labels(e, gmap, level, data, labels, associations):
    if e.inaxes is not None:
        coords = (int(e.ydata // data_shape), int(e.xdata // data_shape))
        neuron = gmap.neurons[coords]
        print(associations)
        if neuron.child_map is not None:
            assc = associations[coords[0]][coords[1]]
            interactive_plot_with_labels(neuron.child_map, dataset=data[assc], labels=labels[assc],
                                         num=str(coords), level=level+1)


def interactive_plot_with_labels(gmap, dataset, labels, num='root', level=1):
    colors = ["#E52B50", "#FFBF00", "#4B0082", "#FBCEB1", "#7FFFD4",
              "#007FFF", "#00FF00", "#9966CC", "#CD7F32", "#89CFF0"]
    sizes = np.arange(0, 60, 6) + 0.5

    mapping = [[list() for _ in range(gmap.map_shape()[1])] for _ in range(gmap.map_shape()[0])]

    _num = "level {} -- parent pos {}".format(level, num)
    fig, ax = plt.subplots(num=_num)
    ax.imshow(__gmap_to_matrix(gmap.weights_map), cmap='bone_r', interpolation='sinc')
    fig.canvas.mpl_connect('button_press_event', lambda event: __plot_child_with_labels(event, gmap, level,
                                                                                        dataset, labels, mapping))
    plt.axis('off')

    for idx, label in enumerate(labels):
        winner_neuron = gmap.winner_neuron(dataset[idx])[0][0]
        r, c = winner_neuron.position
        mapping[r][c].append(idx)

        ax.plot(c*data_shape+data_shape/2, r*data_shape+data_shape/2, 'o', markerfacecolor='None',
                markeredgecolor=colors[label], markersize=sizes[label], markeredgewidth=0.5, label=label)
    legend_handles, legend_labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(legend_labels, legend_handles))
    plt.legend(by_label.values(), by_label.keys(), loc='center left', bbox_to_anchor=(1.1, 0.5), borderaxespad=0.,
               mode='expand', labelspacing=int((gmap.map_shape()[0]/9)*data_shape))
    fig.show()


def mean_data_centroid_activation(ghsom, dataset):
    distances = list()

    for data in dataset:
        _neuron = ghsom
        while _neuron.child_map is not None:
            _gsom = _neuron.child_map
            _neuron = _gsom.winner_neuron(data)[0][0]
        distances.append(_neuron.activation(data))

    distances = np.asarray(a=distances, dtype=np.float32)
    return distances.mean(), distances.std()


def __number_of_neurons(root):
    r, c = root.child_map.weights_map[0].shape[0:2]
    total_neurons = r * c
    for neuron in root.child_map.neurons.values():
        if neuron.child_map is not None:
            total_neurons += __number_of_neurons(neuron)
    return total_neurons


def dispersion_rate(ghsom, dataset):
    used_neurons = dict()
    for data in dataset:
        gsom_reference = ''
        neuron_reference = ''
        _neuron = ghsom
        while _neuron.child_map is not None:
            _gsom = _neuron.child_map
            _neuron = _gsom.winner_neuron(data)[0][0]

            gsom_reference = str(_gsom)
            neuron_reference = str(_neuron)

        used_neurons["{}-{}-{}".format(gsom_reference, neuron_reference, _neuron.position)] = True
    used_neurons = len(used_neurons)

    return __number_of_neurons(ghsom) / used_neurons


if __name__ == '__main__':
    digits = load_digits()

    data = digits.data
    n_samples, n_features = data.shape
    n_digits = len(np.unique(digits.target))
    labels = digits.target

    # mnist = fetch_mldata('MNIST original')
    # data = mnist.data
    # n_samples, n_features = data.shape
    # n_digits = len(np.unique(mnist.target))
    # labels = mnist.target

    print("dataset length: {}".format(n_samples))
    print("features per example: {}".format(n_features))
    print("number of digits: {}\n".format(n_digits))
    ghsom = GHSOM(input_dataset=data, t1=0.1, t2=0.0001, learning_rate=0.15, decay=0.95, gaussian_sigma=1.5)

    print("Training...")
    zero_unit = ghsom.train(epochs_number=15, dataset_percentage=0.50, min_dataset_size=30, seed=0, grow_maxiter=10)

#     print(zero_unit)
    print(mean_data_centroid_activation(zero_unit, data))
    print(dispersion_rate(zero_unit, data))
    interactive_plot_with_labels(zero_unit.child_map, data, labels)
#     plt.show()


[Elapsed Time: 0:00:00] |                                          | (00/1797) 

dataset length: 1797
features per example: 64
number of digits: 10

Training...


[Elapsed Time: 0:00:04] |                                          | (00/1797) 

In [4]:
np.full((2, 3), [255,255,255])

array([[255, 255, 255],
       [255, 255, 255]])

In [8]:
a = data[:10]

In [9]:
a

array([[ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
        15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
        12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
         0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
        10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,
         9.,  0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7.,
        15., 16., 16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,
         0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16.,
        16.,  6.,  0.,  0.,  0.,  0.,  0., 11., 16., 10.,  0.,  0.],
       [ 0.,  0.,  0.,  4., 15., 12.,  0.,  0.,  0.,  0.,  3., 16., 15.,
        14.,  0.,  0.,  0.,  0.,  8., 13.,  8., 16.,  0.,  0.,  0.,  0.,
         1.,  6., 15., 11.,  0.,  0.,  0.,  1.,  8., 13., 15.,  1.,  0.,
         0.,  0.,  9., 16., 16.,  5.,  0.,  0.,  0.,  0.,  

In [10]:
np.array(zero_unit.child_map.winner_neuron(a)).shape#.position

c:\users\ryuki\anaconda3\envs\ghsom\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(2,)

In [ ]:
zero_unit.child_map.

In [7]:
len(zero_unit.child_map.winner_neuron(a)[0])
# zero_unit.child_map.winner_neuron(a)[1]

100

In [15]:
zero_unit.child_map.winner_neuron(a)[0][0]#.position

position (2, 0) -- map dimensions (3, 3, 64) -- input dataset 291 element(s) -- level 0 
	position (0, 0) -- map dimensions (2, 4, 64) -- input dataset 48 element(s) -- level 1 
		position (0, 0) -- map dimensions (2, 5, 64) -- input dataset 10 element(s) -- level 2 
			position (0, 0) -- map dimensions (2, 4, 64) -- input dataset 1 element(s) -- level 3 
			position (1, 0) -- map dimensions (2, 4, 64) -- input dataset 1 element(s) -- level 3 
				position (0, 0) -- map dimensions (3, 2, 64) -- input dataset 1 element(s) -- level 4 
				position (0, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (2, 0) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (2, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (1, 0) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (1, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
			pos

In [6]:
zero_unit.child_map.map_shape()

(3, 3)

In [11]:
zero_unit.child_map.map_shape()

(3, 3)

In [23]:
zero_unit.child_map.weights_map[0].shape

(3, 3, 64)

In [19]:
zero_unit.child_map.weights_map

[array([[[0.00000000e+00, 7.02096055e-03, 1.92459608e+00, 1.06914312e+01,
          1.00771255e+01, 2.35991042e+00, 1.53580144e-01, 1.04741759e-02,
          3.19948509e-04, 3.02547889e-01, 7.43250960e+00, 1.38371025e+01,
          8.65547923e+00, 4.62234411e+00, 6.59325256e-01, 1.76765072e-02,
          0.00000000e+00, 1.25426487e+00, 1.22695700e+01, 9.67879738e+00,
          1.87081940e+00, 4.12810530e+00, 1.87934980e+00, 1.50342233e-01,
          1.50396662e-09, 2.69792073e+00, 1.33353840e+01, 7.77541078e+00,
          3.98286663e+00, 5.78897762e+00, 2.68305705e+00, 2.43234059e-07,
          0.00000000e+00, 3.95874775e+00, 1.32690380e+01, 9.44590852e+00,
          8.34375353e+00, 9.70432122e+00, 4.38856677e+00, 0.00000000e+00,
          7.78066062e-05, 2.41459606e+00, 1.23364158e+01, 8.82206509e+00,
          6.95298540e+00, 1.10585332e+01, 7.22904745e+00, 2.98292647e-01,
          6.46666917e-05, 4.81651630e-01, 9.24222814e+00, 1.13027388e+01,
          8.34312449e+00, 1.03898342e+

In [20]:
zero_unit.child_map.neurons

{(0,
  0): position (0, 0) -- map dimensions (3, 3, 64) -- input dataset 197 element(s) -- level 0 
 	position (0, 0) -- map dimensions (4, 2, 64) -- input dataset 40 element(s) -- level 1 
 		position (0, 0) -- map dimensions (5, 2, 64) -- input dataset 1 element(s) -- level 2 
 			position (0, 0) -- map dimensions (3, 2, 64) -- input dataset 1 element(s) -- level 3 
 			position (0, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 3 
 			position (2, 0) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 3 
 			position (2, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 3 
 			position (1, 0) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 3 
 			position (1, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 3 
 		position (0, 1) -- map dimensions (5, 2, 64) -- input dataset 7 element(s) -- level 2 
 			position (0, 0) -- map dimensions (2, 6, 64) -- input dataset 2 element(s) -- le

In [21]:
zero_unit.child_map.shape

AttributeError: 'GSOM' object has no attribute 'shape'

In [27]:
mapping = [[list() for _ in range(3)]
           for 3 in range(3)]
mapping

SyntaxError: can't assign to literal (<ipython-input-27-c650a35cb0da>, line 2)

In [ ]:
zero_unit.input_dataset.shape

In [13]:
zero_unit.child_map.neurons[(0,1)]

position (0, 1) -- map dimensions (3, 3, 64) -- input dataset 181 element(s) -- level 0 
	position (0, 0) -- map dimensions (4, 2, 64) -- input dataset 24 element(s) -- level 1 
		position (0, 0) -- map dimensions (5, 2, 64) -- input dataset 4 element(s) -- level 2 
			position (0, 0) -- map dimensions (2, 5, 64) -- input dataset 0 element(s) -- level 3 
			position (1, 0) -- map dimensions (2, 5, 64) -- input dataset 1 element(s) -- level 3 
				position (0, 0) -- map dimensions (3, 2, 64) -- input dataset 1 element(s) -- level 4 
				position (0, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (2, 0) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (2, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (1, 0) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
				position (1, 1) -- map dimensions (3, 2, 64) -- input dataset 0 element(s) -- level 4 
			posi